In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/malineni/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.15: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Making `model.base_model.model.vision_tower.vision_model.encoder` require gradients


In [3]:
import os
def read_markdown_files(directory):
    markdown_contents = []
    text_contents = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8", errors='replace') as f:
                        content = f.read()
                        markdown_contents.append(content)
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")
            elif file.endswith(".txt"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8", errors='replace') as f:
                        content = f.read()
                        text_contents.append(content)
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

    return markdown_contents, text_contents

In [4]:
docs_content, text_content = read_markdown_files(os.path.join(os.curdir, "../documentation/marker_pdfs"))
print(docs_content[2])

## U[NIVERSITÄT](http://www.uni-bremen.de) BREMEN

MASTER THESIS

## **Tactile-Based and Cognition-Enabled Manipulation for Real World Assembly Tasks**

*Author:* Arthur NIEDZWIECKI

*Supervisor:* Prof. Dr. h.c. Michael BEETZ, PhD *Second Supervisor:* Dr. rer. nat. Federico RUIZ Ugalde *Advisor:* Gayane KAZHOYAN

*A thesis submitted in fulfillment of the requirements for the degree of Master of Science*

*in the*

[Institute for Artificial Intelligence](http://ai.uni-bremen.de/) [Computer Science](http://www.informatik.uni-bremen.de)

November 11, 2020

- 
- 
- 
- 
- 
- 

Bremen, den 12.11.2020

#### <span id="page-4-0"></span>[UNIVERSITÄT BREMEN](HTTP://WWW.UNI-BREMEN.DE)

## *Abstract*

#### [Faculty 3](http://fb3.uni-bremen.de) [Computer Science](http://www.informatik.uni-bremen.de)

#### Master of Science

#### **Tactile-Based and Cognition-Enabled Manipulation for Real World Assembly Tasks**

#### by Arthur NIEDZWIECKI

Robotic arms are an important asset in efficient automation i

In [5]:
from datasets import Dataset, load_dataset, concatenate_datasets
data1 = {"text": text_content}
datasets1 = Dataset.from_dict(data1)
data2 = {"text": docs_content}
datasets2 = Dataset.from_dict(data2)
datasets = concatenate_datasets([datasets1, datasets2])
EOS_TOKEN = tokenizer.eos_token

In [6]:
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
datasets = datasets.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 13/13 [00:00<00:00, 4775.86 examples/s]


In [7]:
for row in datasets[3:4]["text"]:
    print("=========================")
    print(row)

Universität Bremen Faculty 3, Mathematics and Computer Science

# **Masterthesis**

**Teaching robots manipulation skills with human controlled robots bodies - Spend a day in Virtual Reality inside a robots body** VR-basierte Methoden zur Vermittlung von Manipulationsfähigkeiten in der Robotik - Verbringe den Tag in einem Roboterkörper in einer virtuellen Realität.

> Alina Hawkin 2926383 Supervisor: Prof. Michael Beetz PhD Second Supervisor: Dr. René Weller Advisor: Gayane Kazhoyan Advisor: Andrei Haidu

> > December 7, 2020

# **Declaration of Authorship**

I, Alina Hawkin, declare that this thesis, titled "Teaching robots manipulation skills with human controlled robots bodies -

Spend a day in Virtual Reality inside a robots body" and the work presented in it are my own and has been generated by me as the result of my own original research. I confirm that:

- This work was done wholly or mainly while in candidature for a research degree at the University of Bremen.
- Where any part

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments
from transformers import DataCollatorForLanguageModeling

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasets,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,

        warmup_ratio = 0.1,
        num_train_epochs = 10,

        learning_rate = 5e-5,
        embedding_learning_rate = 5e-6,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8): 100%|██████████| 13/13 [00:02<00:00,  4.70 examples/s]


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 13 | Num Epochs = 10 | Total steps = 30
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 296,198,144/4,000,000,000 (7.40% trained)


Step,Training Loss
1,2.395200
2,2.547100
3,2.103400
4,2.674100
5,2.110700
6,2.402400
7,2.102200
8,2.465200
9,1.850900
10,2.201700


/home/malineni/envs/unsloth/lib/python3.11/site-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

prompts = [
    "what is CRAM action designator,",
    "what is the comprehensive action designator for the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the flanagan motion phases involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the motion constraints involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the framenet elements involved in the task- cut the apple. Give it in json format, no further explanation is needed",
    "what are the object and tools involved in the task- cut the apple. Give it in json format, no further explanation is needed"
]

inputs = tokenizer(
[
    prompts[0]
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 2048*2,
    use_cache = True,
)
thread = Thread(target = model.generate, kwargs = generation_kwargs)
thread.start()

length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width = max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end = "")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end = "")
    pass
pass

## Save the model

In [12]:
model.save_pretrained("gemma_4b_trained")
tokenizer.save_pretrained("gemma_4b_trained")

['gemma_4b_trained/processor_config.json']